In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from datetime import date


from yahoo_fin import stock_info as si

from pandas_datareader import data

In [2]:
def cumm_prod(price_df, start=None, end=None):
    """
    Calculating the cummulative product of the percentage change to normalize the return and be
    able to compare the stocks. 

    """
    if start == None and end == None:
        return_series = (price_df.pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
    else:
        if end == None:
            end = date.today()
        return_series = (price_df.loc[start:end].pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
        
    
    return return_series

def annualized_return(adjClose_return_df, n_years=None):
    """
    n_years needs to represent the period of the adjClose_return_df.
        If it's 1 year worth of cummulative return, you input 1 for the 1y annualized returns.
        If it's 3 years worth of cummulative return, you input 3 for the long period annualized returns.

        (1+total return)pow(1/N) - 1) where N = number of years
    """
    if n_years == None or n_years == 0:
        print("Input How many years does your data covers!")
    else:
        annualized_return = ((1 + adjClose_return_df.tail(1))**(1/n_years)-1) * 100
        return annualized_return

In [3]:
def market_cap_cat(market_cap):
    if float(market_cap[:-1]) > 10:
        market_cap_cat = 'Large'
    elif 2 < float(market_cap[:-1]) < 10:
        market_cap_cat = 'Medium'
    else:
        market_cap_cat = 'Small'
    return market_cap_cat

def earnings_expectations(FPE, TPE):
    if FPE > TPE:
        expect = 'Decrease'
    else:
        expect = 'Increase'
        
    return expect

def CAPM_analysis(ticker, beta, risk_free_rate = 0.0192):
    capm_status = 'TBD'
    
    return_52w = float(si.get_stats(ticker)['Value'][1][:-1])/100
    market_return = float(si.get_stats(ticker)['Value'][2][:-1])/100
    
    capm_ER = risk_free_rate + beta * (market_return - risk_free_rate)

    if return_52w > capm_ER:
        capm_status = 'UnderValued'
    else:
        capm_status = 'OverValued'
    
    return capm_status, capm_ER



def intrinsic_valuation(ticker, eps_ttm, growth_estimate_5y, PE_trailing, target_return, safety_margin=0.5):

    irv_status = 'TBD'

    forecast_eps_at_year_10 = eps_ttm * ( 1 + growth_estimate_5y) ** 9
    forecast_value_at_year_10 = forecast_eps_at_year_10 * PE_trailing

    pv_future_value = forecast_value_at_year_10/(1 + target_return)**9

    pv_forecast_value_safe = pv_future_value * safety_margin

    if float(si.get_live_price(ticker)) > pv_forecast_value_safe:
        irv_status = 'Above Fair Price'
    else:
        irv_status = 'Below Fair Price'

    return irv_status

In [4]:
def get_data(ticker):

    info = {}

    info['ticker'] = ticker
    info['name'] = si.get_quote_data(ticker)['shortName']
    info['sector'] = si.get_company_info(ticker)['Value']['sector']
    info['industry'] = si.get_company_info(ticker)['Value']['industry']

    adj_close = si.get_data(ticker, start_date= str(date.today().year - 10) +'-'+ str(date.today().month) +'-'+ str(date.today().day), end_date=str(date.today()))['adjclose']
    cummulative_prod = cumm_prod(adj_close)
    annu_return_10y = annualized_return(cummulative_prod, 10)[0]

    info['current_price'] = float(si.get_live_price(ticker))
    info['annu_return_10y'] = annu_return_10y

    info['market_cap'] = si.get_quote_table(ticker)['Market Cap']
    info['marketcap_cat'] = market_cap_cat(info['market_cap'])


    info['beta'] = float(si.get_quote_table(ticker)['Beta (5Y Monthly)'])

    info['eps_ttm'] = float(si.get_quote_table(ticker)['EPS (TTM)'])
    info['growth_estimate_5y'] = float(si.get_analysts_info(ticker)['Growth Estimates'][ticker][4][:-1]) / 100

    # if FPE > TPE => Investor expect a decrease in earnings.
    info['pe_forward'] = float(si.get_quote_data(ticker)['forwardPE'])
    info['pe_trailing'] = float(si.get_quote_data(ticker)['trailingPE'])

    info['Earnings_expect'] = earnings_expectations(info['pe_forward'], info['pe_trailing'])
    info['irv_status'] = intrinsic_valuation(ticker, info['eps_ttm'],  info['growth_estimate_5y'], info['pe_trailing'], 0.15)

    info['capm_status'] = CAPM_analysis(ticker, info['beta'])[0]



    return info

In [5]:
panel_data = pd.DataFrame(columns=['ticker', 'name', 'sector', 'industry', 'current_price', 'annu_return_10y', 'marketcap_cat', 'beta', 'eps_ttm', 'growth_estimate_5y', 'pe_forward', 'pe_trailing', 'Earnings_expect', 'irv_status', 'capm_status'])
panel_data

,ticker,name,sector,industry,current_price,annu_return_10y,marketcap_cat,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,Earnings_expect,irv_status,capm_status


In [6]:
sp500_list_10 = si.tickers_sp500()[:4]
ticker_error = {}

In [7]:

# for ticker in sp500_list_10:
#     try:
#         info = get_data(ticker)
#         panel_data.append(info,ignore_index=True)
#     except:
#         pass


In [8]:
panel_data

,ticker,name,sector,industry,current_price,annu_return_10y,marketcap_cat,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,Earnings_expect,irv_status,capm_status


In [9]:
get_data('GOOGL')

{'ticker': 'GOOGL',
 'name': 'Alphabet Inc.',
 'sector': 'Communication Services',
 'industry': 'Internet Content & Information',
 'current_price': 2608.06005859375,
 'annu_return_10y': 23.84385705216705,
 'market_cap': '1.724T',
 'marketcap_cat': 'Small',
 'beta': 1.07,
 'eps_ttm': 112.2,
 'growth_estimate_5y': 0.2756,
 'pe_forward': 19.253359,
 'pe_trailing': 23.245363,
 'Earnings_expect': 'Increase',
 'irv_status': 'Below Fair Price',
 'capm_status': 'UnderValued'}